In [3]:
import pandas as pd 
data = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [4]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [8]:
def clean(data): 
    data = data.drop(["Ticket", "Cabin", "Name", "PassengerId"],axis = 1)
    cols = ["SibSp", "Parch", "Fare", "Age"]
    for col in cols: 
        data[col].fillna(data[col].median(), inplace = True)
        
    data.Embarked.fillna("U", inplace = True)
    return data

In [9]:
data = clean (data)
test = clean(test)

In [10]:
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [27]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data_le = data.copy()
test_le = test.copy()
cols = ['Sex', 'Embarked']
for col in cols: 
    le.fit(data_le[col])
    data_le[col] = le.transform(data_le[col])
    le.fit(test_le[col])
    test_le[col] = le.transform(test_le[col])
    
    print(le.classes_)

['female' 'male']
['C' 'Q' 'S']


In [28]:
data_le.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

y = data_le["Survived"]
X = data_le.drop("Survived", axis = 1)
X_train, X_val , y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42 )

In [30]:
clf = LogisticRegression(random_state = 0, max_iter=1000).fit(X_train, y_train)

In [32]:
prediction = clf.predict(X_val)

In [33]:
from sklearn.metrics import accuracy_score
accuracy_score(y_val, prediction)

0.8100558659217877

In [34]:
submission_preds = clf.predict(test_le)

In [51]:
test = pd.read_csv ("test.csv")
test_ids = test['PassengerId']

In [52]:
df = pd.DataFrame({"PassengerId": test_ids.values, 
                  "Survived" : submission_preds,
                  })

In [53]:
df.to_csv("submission_kaggle.csv", index = False)